# LangChain Expression Language (LCEL)

In [1]:
import openai
import os

In [2]:
openai.api_key = open("openAI_key.txt", "r").read().strip("\n")

In [3]:
!pip install pydantic==1.10.9

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [5]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI(openai_api_key=openai.api_key)
output_parser = StrOutputParser()

In [6]:
chain = prompt | model | output_parser

In [7]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [8]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [9]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(openai_api_key=openai.api_key)
)
retriever = vectorstore.as_retriever()

AttributeError: 'builtin_function_or_method' object has no attribute '__func__'

In [10]:
retriever.get_relevant_documents("where did harrison work?")

NameError: name 'retriever' is not defined

In [ ]:
retriever.get_relevant_documents("what do bears like to eat")

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.schema.runnable import RunnableMap

In [ ]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [ ]:
chain.invoke({"question": "where did harrison work?"})

In [ ]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [ ]:
inputs.invoke({"question": "where did harrison work?"})